In [1]:
import urllib2
import lxml
from lxml import html, etree
import geopandas
import folium
%matplotlib inline
import os
import re
workspace = "geog458\\lab_3\\"

In [2]:
#reference: http://lxml.de/3.1/lxmlhtml.html
#reference: https://stackoverflow.com/questions/37745801/how-can-i-extract-address-from-raw-text-using-nltk-in-python

In [3]:
def saveHtml(link, fileName):
    u = urllib2.urlopen(link)
    filePath = os.path.abspath(fileName + ".html")
    localFile = open(filePath, "w")
    localFile.write(u.read())
    localFile.close()
    print(filePath)
    return filePath

In [4]:
foodBankPath = saveHtml("http://www.needhelppayingbills.com/html/king_county_food_pantries.html","food_bank_data")
homelessPath = saveHtml("https://www.kingcounty.gov/depts/community-human-services/housing/services/homeless-housing/coordinated-entry/access-points.aspx", "homless_shelter_data")

C:\Users\cjms2\geog458\lab_3\food_bank_data.html
C:\Users\cjms2\geog458\lab_3\homless_shelter_data.html


In [5]:
# Reads the local html file
foodBankText = urllib2.urlopen("file:///" + foodBankPath).read()
homelessText = urllib2.urlopen("file:///" + homelessPath).read()

In [6]:
def printPage(doc):
    print etree.tostring(doc, pretty_print = True, method = "html")

In [7]:
foodBankRoot = html.document_fromstring(homelessText)
homelessRoot = html.document_fromstring(homelessText)
printPage(foodBankRoot)
printPage(homelessRoot)



















































































                                <ul class="dropdown-menu">
    <li>
    <div class="yamm-content">
    <div class="row">
    <div class="col-md-4">
    <ul class="list-unstyled">
        <li class="nav-title">Apply for</li>
        <li><a href="/depts/permitting-environmental-review/info.aspx">Permits</a></li>
        <li><a href="/audience/employees/careers.aspx">Jobs at King County</a></li>
        <li><a href="/depts/regional-animal-services/license-your-pet/purchase-renew-license.aspx">Pet license</a></li>
        <li class="hidden-link"><a href="/services/business/licenses.aspx">Business license</a></li>
        <li class="hidden-link"><a href="/depts/records-licensing/recorders-office/marriage-licensing.aspx">Marriage license</a></li>
        <li><a href="#" class="toggle-hidden-links"><span class="fa fa-ellipsis-h"></span></a></li>
    </ul>
    <ul class="list-unstyled">
        <li class="nav-tit

In [8]:
homelessPanels = homelessRoot.find_class("panel-accordion-primary")
veteranHome = homelessPanels[5]
del homelessPanels[5]
print(len(homelessPanels))

10


In [9]:
panelBody = homelessPanels[0].find_class("panel-body")
panelLink = panelBody[0].cssselect("a")
print(panelLink[0].text_content())


100 23rd Ave. S., Seattle, WA 98144


In [10]:
homelessAddress = []
for i in range(0,len(homelessPanels)):
    panelBody = homelessPanels[i].find_class("panel-body")
    panelLink = panelBody[0].cssselect("a")
    panelAddress = panelLink[0].text_content()
    homelessAddress.append(panelAddress.encode("utf-8"))

In [11]:
print(homelessAddress)


['100 23rd Ave. S., Seattle, WA 98144', '1200 S. 336th St., Federal Way, WA 98003', '1010 S. 2nd St., Renton, WA 98057', '9600 College Way N. Seattle, WA 98103', '11061 N.E. 2nd St. Bellevue, WA\xc2\xa098004', '2100 24th Ave S, Seattle, WA 98144', '1828 Yale Avenue, Seattle, WA 98101', '915 H Street SE, Auburn, WA 98002', '16225 NE 87th Street, #1, Redmond, WA 98052', '4740 B University Way NE,\xc2\xa0Seattle, WA 98105']


In [12]:
test = veteranHome.cssselect("p")[5].text_content()

In [13]:
def getAddress(text):
    regExpression = "[0-9]+ .+, .+, [A-Z]{2} [0-9]{5}"
    return re.findall(regExpression, test)

In [14]:
print(test)
print(getAddress(test))
print(getAddress("25715 174th Place SE, Covington, WA 98042"))

VA Puget Sound Community Outreach and Housing Services (CHOS) Renton Walk-in Clinic: 419 S 2nd Street, Suite 2, Renton, WA 98057
['419 S 2nd Street, Suite 2, Renton, WA 98057']
['419 S 2nd Street, Suite 2, Renton, WA 98057']
